In [2]:
from bs4 import BeautifulSoup
import json
with open('leetcode_problems_filtered.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def clean_html(text):
    if not isinstance(text, str):
        return text
    return BeautifulSoup(text, 'html.parser').get_text()

def clean_json_html(obj, keys_to_clean):
    if isinstance(obj, dict):
        return {k: clean_json_html(v, keys_to_clean) if k not in keys_to_clean else clean_html(v) for k,v in obj.items()}
    elif isinstance(obj, list):
        return [clean_json_html(i, keys_to_clean) for i in obj]
    else:
        return obj

fields_to_clean = ['description']
cleaned_data = clean_json_html(data, fields_to_clean)
with open('cleaned_file.json', 'w', encoding='utf-8') as out:
    json.dump(cleaned_data, out, ensure_ascii=False, indent=4)


In [7]:
import pandas as pd
import ast

df = pd.read_excel('leetcode_problems.xlsx')

def parse_list(val):
    try:
        result = ast.literal_eval(val)
        if isinstance(result, list):
            return result
    except:
        pass
    return val

if 'tags' in df.columns:
    df['tags'] = df['tags'].apply(parse_list).apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Save back to Excel
df.to_excel('leetcode_problems_cleaned.xlsx', index=False)


In [9]:
import pandas as pd

leetcode_df = pd.read_excel('leetcode_problems_cleaned.xlsx')
gfg_df = pd.read_csv('gfg_problems_part_2.csv')

# Rename columns for consistency
leetcode_df = leetcode_df.rename(columns={
    'title': 'leetcode_title',
    'description': 'leetcode_description',
    'tags': 'leetcode_tags'
})

gfg_df = gfg_df.rename(columns={
    'Problem Name': 'gfg_title',
    'Description': 'gfg_description',
    'Tags': 'gfg_tags'
})
min_len = min(len(leetcode_df), len(gfg_df))
leetcode_df = leetcode_df.iloc[:min_len].reset_index(drop=True)
gfg_df = gfg_df.iloc[:min_len].reset_index(drop=True)

combined_df = pd.concat([
    leetcode_df[['leetcode_title', 'leetcode_description', 'leetcode_tags']],
    gfg_df[['gfg_title', 'gfg_description', 'gfg_tags']]
], axis=1)

# Add last column named 'result' with all zeros
combined_df['result'] = 0



In [10]:
combined_df

,leetcode_title,leetcode_description,leetcode_tags,gfg_title,gfg_description,gfg_tags,result
0,Two Sum,Given an array of integers nums and an integer...,"Array, Hash Table",Remove common characters and concatenate,Given two strings s1 and s2. Modify both the s...,"Strings, Data Structures",0
1,Add Two Numbers,You are given two non-empty linked lists repre...,"Linked List, Math, Recursion",Count the nodes at distance K from leaf,Given a binary tree with n nodes and a non-neg...,"Tree, Data Structures",0
2,Zigzag Conversion,"The string ""PAYPALISHIRING"" is written in a zi...",String,Number of pairs,Given two arrays arr1[] and arr2[]. Find numbe...,"Arrays, Sorting, Data Structures, Algorithms",0
3,Reverse Integer,"Given a signed 32-bit integer x, return x with...",Math,Max distance between same elements,"Given an array arr[], the task is to find the ...","Arrays, Hash, Data Structures",0
4,Regular Expression Matching,"Given an input string s and a pattern p, imple...","String, Dynamic Programming, Recursion",Jump Game,"Given a array of positive integers arr, where ...","Arrays, Dynamic Programming, Greedy, Data Stru...",0
...,...,...,...,...,...,...,...
569,Jewels and Stones,You're given strings jewels representing the t...,"Hash Table, String",Find k-th smallest element in given n ranges,"Given n ranges of the form [p, q] which denote...","Searching, Sorting, Algorithms",0
570,Basic Calculator III,NaN,"Math, String, Stack, Recursion",Simple Interest,"Given three integers p, r and t, denoting Prin...","Mathematical, Algorithms",0
571,Sliding Puzzle,"On an 2 x 3 board, there are five tiles labele...","Array, Dynamic Programming, Backtracking, Brea...",Smallest divisible number,"Given a number N, find an integer denoting the...","Mathematical, Algorithms",0
572,Minimize Max Distance to Gas Station,NaN,"Array, Binary Search",Clone an Undirected Graph,Given a connected undirected graph with n node...,"Graph, DFS, BFS",0


In [11]:
combined_df.to_excel('combined_problems_with_result.xlsx', index=False)


In [12]:
import pandas as pd
df1 = pd.read_excel('combined_problems_with_result.xlsx')
df2 = pd.read_excel('full duplicate problems (1).xlsx')

df2.columns = [
    'leetcode_title',
    'leetcode_description',
    'leetcode_tags',
    'gfg_title',
    'gfg_description',
    'gfg_tags'
]
df2['result'] = 1

combined_df = pd.concat([df1, df2], ignore_index=True)
combined_df.to_excel('final_combined.xlsx', index=False)
